In [1]:

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import random

In [2]:
# Function to get error code based on the error message
def get_error_code(error_message):
    if 'It looks like you’re a UScellular billed customer. To utilize Quick Pay for our billed customers' in error_message:
        return 1
    elif 'Phone number not valid' in error_message:
        return 3
    elif 'This number is no longer in service' in error_message:
        return 2
    elif 'Please verify you are a human before continuing' in error_message:
        return 4    
    elif 'No errors, it is a prepaid number' in error_message:
        return 0
    else:
        return -1  # Unknown error message
csv_file_path = 'uscs_query_result.csv'

In [3]:
# # Read phone numbers from file and remove country code
# with open('test_panel_usus.txt', 'r') as file:
#     phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file]
# len(phone_numbers)

# Read phone numbers from file and remove country code
with open('test_panel_usus.txt', 'r') as file:
    phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file][64:]
phone_numbers[0]

'4022147803'

In [5]:


options = webdriver.ChromeOptions()
options.headless = True

results = []
consecutive_error_count = 0
current_index = 0  # Initialize the index to keep track of the current phone number

def init_driver():
    """Initialize the Chrome WebDriver."""
    driver = webdriver.Chrome(options=options)
    driver.get('https://prepaid.uscellular.com/refill/dashboard')
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    ).click()
    return driver

# Initialize the WebDriver for the first time
driver = init_driver()


while current_index < len(phone_numbers):
    try:
        phone_number = phone_numbers[current_index]
        print(f"Processing phone number {phone_number} at index {current_index}")

        # phone number input field
        phone_input_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "phone"))
        )
        phone_input_field.click()  # Focus on the input field

        # Clear the input field
        phone_input_field.send_keys(Keys.CONTROL + "a")
        phone_input_field.send_keys(Keys.DELETE)
       
        # Clear the input field and input phone number
        # phone_input_field.clear()
        for char in phone_number:
            phone_input_field.send_keys(char)
            time.sleep(random.uniform(0.3, 1))  # Small delay between each character

        # default error message
        # error_message = "No errors, it is a prepaid number"
        # error_code = get_error_code(error_message)  # Get error code

        # Check for an error message, wait up to 5 seconds
        try:
            error_message_xpath = "/html/body/div[1]/main/div[1]/div/div[2]/div/div[2]/div[1]/div/div[2]/div/div/div/div/form/div[1]/div[1]/div[2]"
            error_message_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, error_message_xpath))
            )
            if error_message_element.text.strip():  # If there's text in the error element
                print(error_message_element.text)
                error_message = error_message_element.text
                error_code = get_error_code(error_message)  # Update error code
        except TimeoutException:
            # error_code == 0
            # error_message = "No errors, it is a prepaid number"
            # print("No errors, it is a prepaid number")
            print('Timeout')
            current_index -= 1
            # If no error message appeared within 5 seconds, assume no error
            pass

        # Reset consecutive error count if no error occurred
        if error_code == 0:
            consecutive_error_count = 0
            current_index += 1  # Increment the index after processing

        else:
            consecutive_error_count += 1
            if consecutive_error_count == 2:  # If three consecutive errors, reset
                driver.quit()
                time.sleep(random.uniform(20, 30))
                driver = init_driver()
                consecutive_error_count = 0
                current_index += 1  # Increment the index after processing
            else:
                current_index += 1
       
        results.append((phone_number, error_code))
        print(phone_number, error_code, error_message)


    except Exception as e:
        print(f"An error occurred: {e}. Retrying the current phone number.")
        # current_index -= 1
        driver.refresh()





# Clean up
driver.quit()



Processing phone number 4022147803 at index 0
It looks like you’re a UScellular billed customer. To utilize Quick Pay for our billed customers Click Here
4022147803 1 It looks like you’re a UScellular billed customer. To utilize Quick Pay for our billed customers Click Here
Processing phone number 3095076943 at index 1
It looks like you’re a UScellular billed customer. To utilize Quick Pay for our billed customers Click Here
3095076943 1 It looks like you’re a UScellular billed customer. To utilize Quick Pay for our billed customers Click Here
Processing phone number 4237216670 at index 2


Exception ignored in: <function Service.__del__ at 0x00000260716F0820>
Traceback (most recent call last):
  File "C:\Users\ericluo\AppData\Roaming\Python\Python39\site-packages\selenium\webdriver\common\service.py", line 185, in __del__
    self.stop()
  File "C:\Users\ericluo\AppData\Roaming\Python\Python39\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\ericluo\AppData\Roaming\Python\Python39\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "c:\ProgramData\Anaconda3\lib\urllib\request.py", line 214, in urlopen
    return opener.open(url, data, timeout)
  File "c:\ProgramData\Anaconda3\lib\urllib\request.py", line 517, in open
    response = self._open(req, data)
  File "c:\ProgramData\Anaconda3\lib\urllib\request.py", line 534, in _open
    result = self._call_chain(self.handle_open, protocol, protocol

Phone number not valid
4237216670 3 Phone number not valid
Processing phone number 9209051343 at index 3
It looks like you’re a UScellular billed customer. To utilize Quick Pay for our billed customers Click Here
9209051343 1 It looks like you’re a UScellular billed customer. To utilize Quick Pay for our billed customers Click Here
Processing phone number 4172242561 at index 4


KeyboardInterrupt: 

In [6]:
df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
df_results.drop_duplicates(inplace=True)
df_results

,Phone Number,Error Code
0,4022147803,1
1,3095076943,1
2,4237216670,3
3,9209051343,1


In [34]:
import os

csv_file_path = 'uscs_query_result.csv'

if os.path.exists(csv_file_path):
    df_existing = pd.read_csv(csv_file_path)
    
    last_index = df_existing.index[-1] if not df_existing.empty else -1

    df_results.index = range(last_index + 1, last_index + 1 + len(df_results))
    
    # Append the new results to the existing DataFrame
    df_final = df_existing.append(df_results)
    df_final = df_final.iloc[last_index + 1:,1:]
else:
    df_final = df_results
    df_final.index = range(len(df_final))

# Output to CSV, without overwriting existing data and with continuous indexing
df_final.to_csv(csv_file_path, index_label='Index', mode='w' if not os.path.exists(csv_file_path) else 'a', header=not os.path.exists(csv_file_path))

print(f"Results have been appended to {csv_file_path}")


Results have been appended to uscs_query_result.csv


C:\Users\ericluo\AppData\Local\Temp\ipykernel_21368\3633805556.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_existing.append(df_results)
